In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
DATASET_PUBMED_RCT_DIR = './gdrive/Shareddrives/DATASETS/PUBMED_RCT/'
OUTPUT_MODEL_DIR = './gdrive/Shareddrives/MODELS/'

In [ ]:
!pip install transformers -q
!pip install sentence_transformers -q
!pip install datasets -q

     |████████████████████████████████| 4.7 MB 8.0 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 596 kB 74.0 MB/s 
     |████████████████████████████████| 85 kB 3.6 MB/s 
     |████████████████████████████████| 1.3 MB 15.1 MB/s 
     |████████████████████████████████| 365 kB 8.6 MB/s 
     |████████████████████████████████| 212 kB 69.2 MB/s 
     |████████████████████████████████| 141 kB 69.1 MB/s 
     |████████████████████████████████| 115 kB 75.7 MB/s 
     |████████████████████████████████| 127 kB 74.6 MB/s 


In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_parquet(DATASET_PUBMED_RCT_DIR + 'train.parquet')
df_dev = pd.read_parquet(DATASET_PUBMED_RCT_DIR + 'dev.parquet')
df_test = pd.read_parquet(DATASET_PUBMED_RCT_DIR + 'test.parquet')

In [ ]:
df_train.columns

Index(['pmid', 'label', 'sentence', 'label_id'], dtype='object')

In [ ]:
df_train['label_id'].unique()

array([0, 1, 2, 3, 4])

In [ ]:
df_train[['label', 'label_id']].drop_duplicates()

,label,label_id
0,objective,0
1,methods,1
6,results,2
11,conclusions,3
12,background,4


In [ ]:
# same text but with different labels
print(f'{len(df_train.index)}')
mask = df_train.groupby('sentence')['label_id'].transform('nunique') > 1
df_train = df_train[~mask].copy()
print(f'{len(df_train.index)}')

180040
179892


In [ ]:
# same text but with different labels
print(f'{len(df_dev.index)}')
mask = df_dev.groupby('sentence')['label_id'].transform('nunique') > 1
df_dev = df_dev[~mask].copy()
print(f'{len(df_dev.index)}')

30212
30212


In [ ]:
# same text but with different labels
print(f'{len(df_test.index)}')
mask = df_test.groupby('sentence')['label_id'].transform('nunique') > 1
df_test = df_test[~mask].copy()
print(f'{len(df_test.index)}')

30135
30122


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("nreimers/MiniLM-L6-H384-uncased")

In [ ]:
def preprocess_function(examples):
  tokenized_sentence = tokenizer(examples["sentence"], truncation=True)
  tokenized_sentence['label'] = examples['label_id']
  return tokenized_sentence

In [ ]:
from datasets import Dataset

dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_dev)
dataset_test = Dataset.from_pandas(df_test)

In [ ]:
dataset_train = dataset_train.map(preprocess_function)
dataset_val = dataset_val.map(preprocess_function)
dataset_test = dataset_test.map(preprocess_function)

  0%|          | 0/179892 [00:00<?, ?ex/s]

  0%|          | 0/30212 [00:00<?, ?ex/s]

  0%|          | 0/30122 [00:00<?, ?ex/s]

In [ ]:
label2id = {'objective': 0, 'methods': 1, 'results': 2, 'conclusions': 3, 'background': 4}
id2label = {0: 'objective', 1: 'methods', 2: 'results', 3: 'conclusions', 4: 'background'}

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

# Download configuration from huggingface.co and cache.
config = AutoConfig.from_pretrained("nreimers/MiniLM-L6-H384-uncased", num_labels=5, id2label=id2label, label2id=label2id)
model = AutoModelForSequenceClassification.from_config(config)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=OUTPUT_MODEL_DIR + 'pubmed_rct_minilm_classification/checkpoint',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_total_limit=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: pmid, label_id, sentence, __index_level_0__. If pmid, label_id, sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 179892
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 224880


Epoch,Training Loss,Validation Loss
1,0.546100,0.501094
2,0.485400,0.462247
3,0.456000,0.466374
4,0.412900,0.466246
5,0.370400,0.474228
6,0.353000,0.490678
7,0.317100,0.517969
8,0.295100,0.552736
9,0.289400,0.611895
10,0.266600,0.676524


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: pmid, label_id, sentence, __index_level_0__. If pmid, label_id, sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30212
  Batch size = 16
Saving model checkpoint to ./gdrive/Shareddrives/MODELS/pubmed_rct_minilm_classification/checkpoint/checkpoint-11244
Configuration saved in ./gdrive/Shareddrives/MODELS/pubmed_rct_minilm_classification/checkpoint/checkpoint-11244/config.json
Model weights saved in ./gdrive/Shareddrives/MODELS/pubmed_rct_minilm_classification/checkpoint/checkpoint-11244/pytorch_model.bin
tokenizer config file saved in ./gdrive/Shareddrives/MODELS/pubmed_rct_minilm_classification/checkpoint/checkpoint-11244/tokenizer_config.json
Special tokens file saved in ./gdrive/Shareddrives/MODELS/pubmed_rct_minilm_

TrainOutput(global_step=224880, training_loss=0.27584971080509973, metrics={'train_runtime': 9894.4827, 'train_samples_per_second': 363.621, 'train_steps_per_second': 22.728, 'total_flos': 2.035533782180328e+16, 'train_loss': 0.27584971080509973, 'epoch': 20.0})

In [ ]:
predictions = trainer.predict(dataset_test)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: pmid, label_id, sentence, __index_level_0__. If pmid, label_id, sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 30122
  Batch size = 16


In [ ]:
from datasets import load_metric
import numpy as np

def compute_metrics(eval_pred):
  metric_acc = load_metric("accuracy")
  metric_p = load_metric("precision")
  metric_r = load_metric("recall")
  metric_f1_macro = load_metric("f1")
  metric_f1_micro = load_metric("f1")
    
  preds = eval_pred.predictions
  labels = eval_pred.label_ids
  predictions = np.argmax(preds, axis=-1)

  accuracy = metric_acc.compute(predictions=predictions, references=labels)
  #precision = metric_p.compute(predictions=predictions, references=labels)
  #recall = metric_r.compute(predictions=predictions, references=labels)
  f1_micro = metric_f1_micro.compute(predictions=predictions, references=labels, average="micro")
  f1_macro = metric_f1_micro.compute(predictions=predictions, references=labels, average="macro")
  
  return {'accuracy': accuracy, 'f1_macro': f1_macro, 'f1_micro': f1_micro}

In [ ]:
compute_metrics(predictions)

{'accuracy': {'accuracy': 0.824048867937056},
 'f1_macro': {'f1': 0.7654031347760255},
 'f1_micro': {'f1': 0.824048867937056}}

In [ ]:
OUTPUT_MODEL_DIR = './gdrive/Shareddrives/MODELS/'

model.save_pretrained(OUTPUT_MODEL_DIR + 'pubmed_rct_minilm_classification/model')

Configuration saved in ./gdrive/Shareddrives/MODELS/pubmed_rct_minilm_classification/model/config.json
Model weights saved in ./gdrive/Shareddrives/MODELS/pubmed_rct_minilm_classification/model/pytorch_model.bin
